## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [113]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-cool-experiment', tags={}>]

In [6]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [129]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [130]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d75a57a0b09b4fdf8a761b5d1e6c0bf9, rmse: 6.3040
run id: be4181c0467a4f0aa5f4bba1c916b76a, rmse: 6.3591
run id: 86ecd568107644cf8f674e4cc474570f, rmse: 6.3676
run id: 6f82b26b9daa428ca6f9ee71b755dc8a, rmse: 6.4038
run id: a5e630d87d4e4b6386d2412a1b5e53aa, rmse: 6.4128
run id: c589dbda396f457bb60eaa0ff0a375a7, rmse: 6.4640
run id: 92f435c1da364443ab936eae01165d31, rmse: 6.4700
run id: ce2f8793e59a47d8829ffbea07b914ce, rmse: 6.5488
run id: 5eeced395072424c9cee646a8b271402, rmse: 6.5505
run id: 46fb625e4cf84f5e8f01b10212303806, rmse: 6.6023


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [28]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [101]:
run_id = "7e3634992ed249478323288c1e723eb3"
model_uri = f"runs:/{run_id}/model"
mv = mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/25 23:50:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 9
Created version '9' of model 'nyc-taxi-regressor'.


In [136]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 8, stage: Staging
version: 3, stage: None
version: 9, stage: Production


In [138]:
# Promote new model to staging
new_stage = "Staging"
mv = client.transition_model_version_stage(
    name=model_name,
    version=mv.version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653515401371, current_stage='Staging', description='The model version 9 was transitioned to Staging on 2022-05-25', last_updated_timestamp=1653517383286, name='nyc-taxi-regressor', run_id='7e3634992ed249478323288c1e723eb3', run_link=None, source='./mlruns/1/7e3634992ed249478323288c1e723eb3/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=9>

In [139]:
from datetime import datetime

date = datetime.today().date()

mv = client.update_model_version(
    name=model_name,
    version=mv.version,
    description=f"The model version {mv.version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653515401371, current_stage='Staging', description='The model version 9 was transitioned to Staging on 2022-05-26', last_updated_timestamp=1653517386603, name='nyc-taxi-regressor', run_id='7e3634992ed249478323288c1e723eb3', run_link=None, source='./mlruns/1/7e3634992ed249478323288c1e723eb3/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=9>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [141]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    print("Model: ", f"models:/{name}/{stage}")
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [142]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [143]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/karim/workspace/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [144]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [145]:
X_test = preprocess(df, dv)

In [146]:
target = "duration"
y_test = df[target].values

In [147]:
print(model_name)

nyc-taxi-regressor


In [149]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

Model:  models:/nyc-taxi-regressor/Production
[00:25:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 15.8 s, sys: 65.3 ms, total: 15.8 s
Wall time: 1.48 s


{'rmse': 6.250963441118867}

In [150]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

Model:  models:/nyc-taxi-regressor/Staging
CPU times: user 122 ms, sys: 6.44 ms, total: 129 ms
Wall time: 128 ms


{'rmse': 6.981846189522709}

In [152]:
client.transition_model_version_stage(
    name=model_name,
    version=mv.version,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1653515401371, current_stage='Production', description='The model version 9 was transitioned to Staging on 2022-05-26', last_updated_timestamp=1653517607274, name='nyc-taxi-regressor', run_id='7e3634992ed249478323288c1e723eb3', run_link=None, source='./mlruns/1/7e3634992ed249478323288c1e723eb3/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=9>